In [1]:
%load_ext autoreload
%load_ext autotime

time: 374 µs (started: 2023-03-06 18:44:30 -08:00)


In [2]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 1.76 s (started: 2023-03-06 18:44:30 -08:00)


In [3]:
import dask.dataframe as dd
import numpy as np
import pandas as pd


time: 660 ms (started: 2023-03-06 18:44:32 -08:00)


In [4]:
%autoreload 2

from wisio.recorder import RecorderAnalyzer

log_dir = "/p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet"

recorder_analyzer = RecorderAnalyzer(
    cluster_manager_args=dict(
        force_local=True,
        cluster_settings=dict(
            cores=4,
            dashboard_port=3666,
            local_directory="/var/tmp/dask-recorder",
            log_file="%J.log",
            worker_queue="pdebug"
        )
    ),
    working_dir='.recorder',
    debug=True
) 

recorder_analyzer

time: 2.46 s (started: 2023-03-06 18:44:33 -08:00)


In [5]:
[client.dashboard_link for client in recorder_analyzer.cluster_manager.clients]

['http://192.168.131.2:3666/status']

time: 3 ms (started: 2023-03-06 18:44:35 -08:00)


In [6]:
%autoreload 2

res = recorder_analyzer.analyze_parquet(log_dir=log_dir)

time: 28.2 s (started: 2023-03-06 18:44:36 -08:00)


In [8]:
views, bottlenecks = res

time: 994 µs (started: 2023-03-06 18:47:42 -08:00)


In [9]:
views[('file_id',)]['bottleneck_view'].head()

,trange,file_id,proc_id,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,...,sequential_time,random_time,sequential_size,random_size,sequential_count,random_count,duration_score,duration_cut,duration_pero,duration_perr
2062,1,7639450094153155784,5901139297808997828,4,0,0.165197,2993,2,172352,513959428,...,0.164976,0,513959428,0,2986,0,5,0.5,0.046074,0.004646
2063,1,7639450094153155784,7500532995116292548,6,0,0.068847,408,5760,175232,57919360,...,0.066361,0,57919360,0,398,0,5,0.5,0.019202,0.001936
2064,2,7639450094153155784,5901139297808997828,1,0,0.134732,3786,172352,172352,652524672,...,0.134732,0,652524672,0,3786,0,5,0.5,0.037577,0.003789
2065,2,7639450094153155784,7500532995116292548,5,0,0.003474,137,1,172352,23267521,...,0.003106,0,23267521,0,136,0,5,0.5,0.000969,0.000098
2066,3,7639450094153155784,5901139297808997828,1,0,0.135458,3803,172352,172352,655454656,...,0.135458,0,655454656,0,3803,0,5,0.5,0.037780,0.003809


time: 214 ms (started: 2023-03-06 18:47:42 -08:00)


In [10]:
bottlenecks[('trange',)]

{}

time: 2.86 ms (started: 2023-03-06 18:47:48 -08:00)


In [11]:
bottlenecks[('file_id',)]

{'/p/gpfs1/iopp/temp/montage/2854964/25/run/ngc3372.fits': {'llc': {'duration_sum': 3.5825827,
   'index_count': 153464,
   'size_min': 0,
   'size_max': 18440,
   'size_sum': 2829513124,
   'func_id': ['fclose', 'fopen64', 'fread', 'fwrite'],
   'read_time': 3.5823433697223663,
   'write_time': 0.10802439786493778,
   'metadata_time': 0.0005902999982936308,
   'read_size': 2735911684,
   'write_size': 93601440,
   'metadata_size': 0,
   'read_count': 148372,
   'write_count': 5076,
   'metadata_count': 16,
   'data_count': 153448,
   'data_size': 2829513124,
   'data_time': 3.5823433697223663,
   'sequential_time': 3.5823433697223663,
   'random_time': 0,
   'sequential_size': 2829513124,
   'random_size': 0,
   'sequential_count': 153448,
   'random_count': 0,
   'duration_score': 96,
   'duration_cut': 12.0,
   'duration_pero': 0.9992021,
   'duration_perr': 0.10074785},
  'proc_name': {'mAddMPI-lassen578-24': {'llc': {'duration_sum': 0.0983191,
     'index_count': 1271,
     'size_

time: 77.9 ms (started: 2023-03-06 18:47:50 -08:00)
